## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-10-13-14-03 +0000,nyt,"Many Residents in Minneapolis, Tempered by ICE...",https://www.nytimes.com/2026/02/10/us/twin-cit...
1,2026-02-10-13-11-11 +0000,nyt,"Lutnick, Wasserman and others caught up in the...",https://www.nytimes.com/2026/02/10/business/de...
2,2026-02-10-13-03-45 +0000,nyt,How the Israeli President’s Visit to Australia...,https://www.nytimes.com/2026/02/10/world/austr...
3,2026-02-10-13-03-16 +0000,bbc,Ministers told to not publish their own Mandel...,https://www.bbc.com/news/articles/cm28yzxwp2vo...
4,2026-02-10-13-01-37 +0000,nypost,Ex-FBI agent describes what ‘really took my br...,https://nypost.com/2026/02/10/us-news/nancy-gu...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2436/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
132,trump,30
228,new,21
12,epstein,19
326,canada,14
100,year,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
161,2026-02-10-01-54-00 +0000,wsj,Trump Threatens to Block Opening of New Bridge...,https://www.wsj.com/world/americas/trump-threa...,93
96,2026-02-10-09-57-46 +0000,nypost,Trump says he will block US-Canada Bridge unle...,https://nypost.com/2026/02/10/us-news/trump-sa...,91
48,2026-02-10-11-32-07 +0000,nypost,Trump told Palm Beach police chief ‘thank good...,https://nypost.com/2026/02/10/us-news/trump-to...,90
319,2026-02-09-15-32-32 +0000,nypost,"Ghislaine Maxwell offers to clear Trump, Clint...",https://nypost.com/2026/02/09/us-news/ghislain...,82
195,2026-02-09-23-15-50 +0000,cbc,Trump threatens to block opening of new bridge...,https://www.cbc.ca/news/canada/trump-gordie-ho...,81


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
161,93,2026-02-10-01-54-00 +0000,wsj,Trump Threatens to Block Opening of New Bridge...,https://www.wsj.com/world/americas/trump-threa...
40,77,2026-02-10-11-41-00 +0000,wsj,"The U.S. economy looks healthy, but it crossed...",https://www.wsj.com/finance/stocks/healthcare-...
114,70,2026-02-10-08-32-51 +0000,nypost,Nancy Guthrie was supposed to go to pal’s hous...,https://nypost.com/2026/02/10/us-news/nancy-gu...
48,60,2026-02-10-11-32-07 +0000,nypost,Trump told Palm Beach police chief ‘thank good...,https://nypost.com/2026/02/10/us-news/trump-to...
183,52,2026-02-10-00-06-00 +0000,wsj,Soaring profits and stocks funnel more of GDP ...,https://www.wsj.com/economy/jobs/capital-labor...
76,50,2026-02-10-10-34-24 +0000,nypost,Bad Bunny wipes Instagram after Super Bowl LX ...,https://nypost.com/2026/02/10/us-news/bad-bunn...
314,40,2026-02-09-16-00-19 +0000,cbc,Canada's major airlines wind down flights to C...,https://www.cbc.ca/news/canada/travel-to-cuba-...
326,33,2026-02-09-15-02-10 +0000,bbc,Scottish Labour leader Anas Sarwar calls on Ke...,https://www.bbc.com/news/videos/cqj9l519v70o?a...
267,33,2026-02-09-19-03-07 +0000,nypost,SF teachers strike for first time in 50 years ...,https://nypost.com/2026/02/09/us-news/san-fran...
169,31,2026-02-10-01-25-52 +0000,latimes,Federal judge strikes down California mask ban...,https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
